In [25]:
""" #Plot the learning curve
from flaml.automl.data import get_output_from_log

time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history =  get_output_from_log(filename='flaml_log_basic.log', time_budget=60)


import matplotlib.pyplot as plt
import numpy

plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Loss (RSME)")
plt.step(time_history,np.array(best_valid_loss_history), where="post")
plt.show() """

' #Plot the learning curve\nfrom flaml.automl.data import get_output_from_log\n\ntime_history, best_valid_loss_history, valid_loss_history, config_history, metric_history =  get_output_from_log(filename=\'flaml_log_basic.log\', time_budget=60)\n\n\nimport matplotlib.pyplot as plt\nimport numpy\n\nplt.title("Learning Curve")\nplt.xlabel("Wall Clock Time (s)")\nplt.ylabel("Validation Loss (RSME)")\nplt.step(time_history,np.array(best_valid_loss_history), where="post")\nplt.show() '

In [26]:
import pandas as pd
import numpy as np
from utils.personal_utils import *
from create_prepared_data_tsfresh import *

In [27]:
all_ds = {
    'AppliancesEnergy':   '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts',
    'NewsTitleSentiment': '/home/sim/Desktop/TS Extrinsic Regression/data/NewsTitleSentiment_TRAIN.ts',
    'BenzeneConcentration':'/home/sim/Desktop/TS Extrinsic Regression/data/BenzeneConcentration_TRAIN.ts',
    'BeijingPM25Quality': '/home/sim/Desktop/TS Extrinsic Regression/data/BeijingPM25Quality_TRAIN.ts',

    'IEEEPPG':            '/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TRAIN.ts',
    'FloodModeling1':     '/home/sim/Desktop/TS Extrinsic Regression/data/FloodModeling1_TRAIN.ts',
    'HouseholdPowerConsumption1': '/home/sim/Desktop/TS Extrinsic Regression/data/HouseholdPowerConsumption1_TRAIN.ts',
    'Covid3Month':              '/home/sim/Desktop/TS Extrinsic Regression/data/Covid3Month_TRAIN.ts' 
}

In [1]:
import pandas as pd
import numpy as np

In [9]:
from autogluon.tabular import TabularPredictor

# Specify the path to the saved model directory
model_path = '/home/sim/Desktop/All-Logs_agluon/Test/test_run'

# Load the model
predictor = TabularPredictor.load(model_path)

ds_name = 'AppliancesEnergy'
features_path = '/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + '/'
test_features = pd.read_parquet(features_path + f'NONE_NONE_{ds_name}_features_TEST')
print(predictor.predict(test_features))
prediction_true = np.load('/home/sim/Desktop/All-Logs_agluon/Test/test_run/predictions.npy')
print(prediction_true)



0     17.134245
1     15.838378
2     10.849128
3     11.105565
4     15.165165
5     12.991673
6     18.859991
7     12.126429
8     15.322295
9     11.490843
10    17.471922
11    13.480631
12    12.298971
13    13.195881
14    14.009985
15    16.923445
16    12.592018
17    15.375562
18    10.259102
19    11.992506
20    11.211722
21    15.008444
22    11.370363
23    14.259556
24    16.449863
25    11.703172
26    14.483973
27    18.610455
28    12.913260
29    12.395144
30    17.138468
31    13.842776
32    13.312163
33    16.537073
34    11.228756
35    16.352789
36    11.060820
37    14.962638
38    18.436823
39    11.440249
40    12.503656
41    11.097223
Name: target, dtype: float32
[17.134245  15.838378  10.849128  11.105565  15.165165  12.9916725
 18.859991  12.126429  15.322295  11.490843  17.471922  13.480631
 12.298971  13.195881  14.009985  16.923445  12.592018  15.375562
 10.259102  11.992506  11.211722  15.008444  11.370363  14.259556
 16.449863  11.703172  14.483973  

NameError: name 'pre' is not defined

In [ ]:
# Test if saved .pkl is working!
import pickle
with open('/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/flaml_model.pkl', 'rb') as model_file:
    flaml_model = pickle.load(model_file)


# Load the predictions
prediction_true = np.load('/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/predictions.npy')

tsandf, features = load_and_prepare_everything(all_ds['AppliancesEnergy'], 'dct', 0.5)


# Only 
file_path = '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

selected_features = features[lines]

prediction_pkl = flaml_model.predict(selected_features)

print(prediction_true)
print(prediction_pkl)
print(np.array_equal(prediction_pkl, prediction_true))

In [42]:
# Test if saved .pkl is working!
import pickle
with open('/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/flaml_model.pkl', 'rb') as model_file:
    flaml_model = pickle.load(model_file)


# Load the predictions
prediction_true = np.load('/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/predictions.npy')

tsandf, features = load_and_prepare_everything(all_ds['AppliancesEnergy'], 'dct', 0.5)


# Only 
file_path = '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

selected_features = features[lines]

prediction_pkl = flaml_model.predict(selected_features)

print(prediction_true)
print(prediction_pkl)
print(np.array_equal(prediction_pkl, prediction_true))


66it [00:01, 60.51it/s] 
Feature Extraction: 100%|██████████| 30/30 [00:10<00:00,  2.98it/s]


[17.40866404 16.08758159 11.58181119 12.63613691 15.49786128 12.50503084
 18.73428551 14.34195033 15.65821943 13.20204821 18.73780325 14.5782942
 12.93648353 11.61012514 10.86518501 18.46632804 14.97628939 13.58770037
  9.54824379 11.66188595 11.87137519 18.14336381 12.11094599 11.36045666
 15.58979133 12.94519136 13.9598636  16.37746201 11.98862467 14.76332943
 17.75137696 13.47867751 11.25503249 14.17218026 10.45279926 17.51902999
 10.08038137 11.42773212 17.72179858 11.54069719 11.92035155 12.86367304]
[17.40866404 16.08758159 11.58181119 12.63613691 15.49786128 12.50503084
 18.73428551 14.34195033 15.65821943 13.20204821 18.73780325 14.5782942
 12.93648353 11.61012514 10.86518501 18.46632804 14.97628939 13.58770037
  9.54824379 11.66188595 11.87137519 18.14336381 12.11094599 11.36045666
 15.58979133 12.94519136 13.9598636  16.37746201 11.98862467 14.76332943
 17.75137696 13.47867751 11.25503249 14.17218026 10.45279926 17.51902999
 10.08038137 11.42773212 17.72179858 11.54069719 11.

In [34]:
# Load single ts
data_path = '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TRAIN.ts'
ts_1 = load_single_ts(data_path, 5, 3)
matrix_appliances = load_datapoint(data_path, 0)

119it [00:02, 47.42it/s]
100%|██████████| 95/95 [00:00<00:00, 500.36it/s]
119it [00:02, 46.02it/s]
100%|██████████| 95/95 [00:00<00:00, 644.70it/s]


In [35]:
# Quantization Testing: Test the returnd dtyps of the transforms

import pandas as pd
import numpy as np

from scipy.fft import dct, idct, fft, ifft
import pywt


print(ts_1.dtype)
ts_1 = ts_1.astype(np.float32)

print(dct(ts_1, norm='ortho',type=2).dtype)
print(fft(ts_1, norm='ortho').real.dtype)
print(pywt.dwt(ts_1, 'db4')[1].dtype)


float64
float32
float32
float32


In [36]:
data_path2 = '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TRAIN.ts'
matrix_ieee = load_datapoint(data_path2, 90)
ts_ie = load_single_ts(data_path2, 5, 3)

119it [00:02, 46.34it/s]
100%|██████████| 95/95 [00:00<00:00, 594.91it/s]
119it [00:02, 47.61it/s]
100%|██████████| 95/95 [00:00<00:00, 475.54it/s]


In [37]:
#Test general shape of compression
dr = 0.4
print(ts_ie[:100])
print(dwt_compress(ts_ie[0:100], dr ))
#print(compute_rmse(ts_ie, dwt_compress(ts_ie, dr )))

[-1.84874111 -1.79521367 -1.72830437 -1.62392587 -1.47806359 -1.36833234
 -1.27332114 -1.27332114 -1.22113189 -1.19704454 -1.11675338 -1.06054957
 -1.00568394 -0.89862906 -0.76882502 -0.68585749 -0.59486084 -0.52360244
 -0.44364583 -0.33792913 -0.25763797 -0.19340505 -0.16396496 -0.08501198
 -0.00472082  0.05148299  0.05148299  0.13177415  0.10233406  0.08761401
  0.08761401  0.07557034  0.05148299  0.08761401  0.1170541   0.13177415
  0.13177415  0.15184694  0.17191973  0.19600708  0.25221089  0.27629823
  0.26425456  0.28834191  0.33250205  0.35926577  0.3846913   0.40877865
  0.43821874  0.43821874  0.46498246  0.48906981  0.53322994  0.55999366
  0.57337552  0.57337552  0.57337552  0.62422659  0.65366668  0.65366668
  0.67775403  0.6897977   0.7192378   0.7192378   0.7192378   0.7473397
  0.77410342  0.81023444  0.83967454  0.8905256   0.86643825  0.85439458
  0.85439458  0.85439458  0.85439458  0.83967454  0.81023444  0.81023444
  0.81023444  0.7861471   0.76072156  0.73395784  0.

TypeError: dwt_compress() missing 4 required positional arguments: 'andDecompress', 'level', 'wavelet', and 'quantization_level'

In [ ]:
# Compute dft on every column of matrix
matrix = matrix_ieee

matrix_compressed = np.zeros(matrix.shape)
for i in range(matrix.shape[1]):
    matrix_compressed[:,i] = dwt_compress(matrix[:,i], 0.65)


compute_rmse_of_datapoint(matrix, matrix_compressed)

#(1.4435339091230222,
#array([3.71996501, 3.38425485, 0.03060634, 0.0288563 , 0.05398704]))

(0.9745457315927168,
 array([2.51989208, 2.29014343, 0.01868709, 0.01885416, 0.02515189]))

Testing calculateCompRatio

In [ ]:
# Testing calculateCompRatio
import gzip

def calculateCompRatio(dataset_array, array_flatdim_comp):

    # Calculate the number of bytes in the raw dataset array
    num_bytes_raw = dataset_array.nbytes
    print("Number of bytes in the raw dataset array:", num_bytes_raw)
    print("Number of the bytes in array_comp before gzip", array_flatdim_comp.nbytes)
    # Flatten the compressed array in column-major order. # Tested it, not need to put .copy() after.
    array_flatdim_1d = array_flatdim_comp.reshape(-1, order='F')
    print("Array flattened", array_flatdim_1d)

    # Convert the flattened array to a byte string
    compressed_data_bytestring = array_flatdim_1d.tobytes()

    print("Size of raw_data after gzip", len(gzip.compress(dataset_array.tobytes())))

    # Compress the byte string using gzip compression
    compressed_data_gzipd = gzip.compress(compressed_data_bytestring, compresslevel=9)
    print("Size of the compressed data after gzip:", len(compressed_data_gzipd))



    # Calculate and return the compression ratio
    compression_ratio = num_bytes_raw / len(compressed_data_gzipd)
    print("Compression ratio:", compression_ratio)
    return compression_ratio


In [ ]:
# Try with easy data. Then try with real time-series.

num_dp = 3
len_ts = 9
num_dim = 4

array = np.arange(num_dp * len_ts * num_dim)
array_comp = np.arange(num_dp * len_ts * num_dim)



array_comp = array_comp.reshape(num_dp * len_ts, num_dim)

array_comp[[1,2,3,5,6,7,9,12,13,14,15,16,15,16,17,20]] = 0


print(array_comp)
calculateCompRatio(array, array_comp)


# Conclusions:
# Values look good! Show that also with zeroing out we reached a little more compression!